In [1]:
import pandas as pd
import keras
!pip install keras-tuner
!pip install tensorflow
from kerastuner import RandomSearch
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 1.3 MB/s eta 0:00:00


<ipython-input-1-cd4ddce39d7a>:5: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


### Preparing train and val dataset for ANN

In [2]:
# Paths to the train and validation datasets
train_data_path = 'train_data.csv'
val_data_path = 'val_data.csv'
test_data_path = 'test_data.csv'

In [3]:
# Load the datasets
train_data = pd.read_csv(train_data_path)
val_data = pd.read_csv(val_data_path)
test_data = pd.read_csv(test_data_path)
print(train_data.shape)
print(val_data.shape)
print(test_data.shape)

(2791, 1981)
(931, 1981)
(931, 1981)


In [4]:
# Splitting the datasets into features (X) and target (y)
X_train = train_data.drop('Output', axis=1)
y_train = train_data['Output']
X_val = val_data.drop('Output', axis=1)
y_val = val_data['Output']
X_test = test_data.drop('Output', axis=1)
y_test = test_data['Output']

In [5]:
# Check the shapes of the datasets
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (2791, 1980)
y_train shape: (2791,)
X_val shape: (931, 1980)
y_val shape: (931,)
X_test shape: (931, 1980)
y_test shape: (931,)


In [6]:
print (y_val)

0      0.080714
1      0.687701
2      0.019128
3      0.191419
4      0.391193
         ...   
926    0.470977
927    0.183726
928    0.029032
929    0.089227
930    0.197472
Name: Output, Length: 931, dtype: float64


### Building ANN model

In [7]:
import numpy as np

In [8]:
# ANN model structure
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units_1', min_value=32, max_value=512, step=32),
                    activation='relu', input_shape=(1980,)))
    model.add(Dropout(rate=hp.Float('dropout_1', min_value=0, max_value=0.5, step=0.1)))
    model.add(Dense(units=hp.Int('units_2', min_value=32, max_value=512, step=32),
                    activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout_2', min_value=0, max_value=0.5, step=0.1)))
    model.add(Dense(1))  # Output layer
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [9]:
# Hyperparameter tuner
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=25,  # Number of trials
    executions_per_trial=5,  # Number of models to train per trial
    directory='my_dir',
    project_name='ann_tuning'
)

# Hyperparameter search
tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Trial 25 Complete [00h 00m 33s]
val_loss: 0.004845475126057863

Best val_loss So Far: 0.001386262266896665
Total elapsed time: 00h 13m 56s


In [10]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the best hyperparameters and train it
best_model = tuner.hypermodel.build(best_hps)

In [11]:
# Combine the train and validation sets for final training
X_combined = np.concatenate((X_train, X_val), axis=0)
y_combined = np.concatenate((y_train, y_val), axis=0)

In [12]:
from sklearn.utils import shuffle

X_combined, y_combined = shuffle(X_combined, y_combined)

In [13]:
# Train the best model on the combined dataset
history = best_model.fit(X_combined, y_combined, epochs=50, batch_size=32, verbose=1)

Epoch 1/50
117/117 [==============================] - 1s 3ms/step - loss: 0.7098
Epoch 2/50
117/117 [==============================] - 0s 3ms/step - loss: 0.0053
Epoch 3/50
117/117 [==============================] - 0s 3ms/step - loss: 0.0037
Epoch 4/50
117/117 [==============================] - 0s 3ms/step - loss: 0.0045
Epoch 5/50
117/117 [==============================] - 0s 3ms/step - loss: 0.0023
Epoch 6/50
117/117 [==============================] - 0s 3ms/step - loss: 0.0020
Epoch 7/50
117/117 [==============================] - 0s 3ms/step - loss: 0.0022
Epoch 8/50
117/117 [==============================] - 0s 3ms/step - loss: 0.0018
Epoch 9/50
117/117 [==============================] - 0s 3ms/step - loss: 0.0019
Epoch 10/50
117/117 [==============================] - 0s 3ms/step - loss: 0.0018
Epoch 11/50
117/117 [==============================] - 0s 4ms/step - loss: 0.0041
Epoch 12/50
117/117 [==============================] - 0s 4ms/step - loss: 0.0026
Epoch 13/50
117/117 [====

In [14]:
# After training, save the best model to an HDF5 file
best_model_path = 'collab_best_ANN_model.h5'  # Replace with your desired path
best_model.save(best_model_path)

print(f"The best model is saved to {best_model_path}")

The best model is saved to collab_best_ANN_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Best model loss and prediction for test set

In [16]:
# Evaluate the model on the test set
test_loss = best_model.evaluate(X_test, y_test, verbose=0)

In [17]:
# Calculate predictions to evaluate other metrics such as R^2 or MAE
y_pred = best_model.predict(X_test)

30/30 [==============================] - 0s 2ms/step


### Getting model and result info

In [18]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

# Assuming y_test and y_pred are already defined
# You can replace them with your actual test and prediction data

# Calculate R^2 score
r2 = r2_score(y_test, y_pred)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)

# Calculate RMSE
rmse = np.sqrt(mse)

# Calculate MAPE
# Assuming y_pred is the array you provided
y_pred = y_pred.flatten()

# Assuming test_loss is defined
print(f"Test Loss: {test_loss}")
print(f"R^2 Score: {r2}")
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")

# Now y_pred is one-dimensional, and you can proceed with calculations like MAPE
# Ensure there are no zero elements in y_test to avoid division by zero in MAPE calculation
if np.any(y_test == 0):
    print("Error: y_test contains zero values, which will lead to division by zero in MAPE calculation.")
else:
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print(f"Mean Absolute Percentage Error: {mape}%")

Test Loss: 0.0006845650495961308
R^2 Score: 0.9906122923210201
Mean Absolute Error: 0.020373078590742496
Mean Squared Error: 0.0006845650111091428
Root Mean Squared Error: 0.026164193301325817
Error: y_test contains zero values, which will lead to division by zero in MAPE calculation.


In [19]:
from keras.models import load_model

# Load the best model
best_model = load_model(best_model_path)

# Get the configuration of the model
config = best_model.get_config()

# Extract the units from the Dense layers
# Adjust the indices according to your model's architecture
dense1_units = config['layers'][1]['config']['units']
dense2_units = config['layers'][3]['config']['units']

# Extract the dropout rate from the Dropout layer
# Adjust the indices if necessary
dropout1_rate = config['layers'][2]['config']['rate']
dropout2_rate = config['layers'][4]['config']['rate']

# Learning rate is part of the optimizer's configuration
learning_rate = best_model.optimizer.learning_rate.numpy()

# Output the extracted values
print(f"Dense Layer 1 Units: {dense1_units}")
print(f"Dense Layer 2 Units: {dense2_units}")
print(f"Dropout Layer 1 Rate: {dropout1_rate}")
print(f"Dropout Layer 2 Rate: {dropout2_rate}")
print(f"Learning Rate: {learning_rate}")

Dense Layer 1 Units: 384
Dense Layer 2 Units: 96
Dropout Layer 1 Rate: 0.0
Dropout Layer 2 Rate: 0.0
Learning Rate: 0.0010000000474974513
